In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import csv
import numpy as np

print('Importing packages (done)')

Importing packages (done)


In [5]:
# Login to Linkedin

# Open Chrome and Access Linkedin login site
driver = webdriver.Chrome("C:/Users/guedri/Downloads/chromedriver_win32/chromedriver.exe")
sleep(2)
url = 'https://www.linkedin.com/login'
driver.get(url)
print('Initializing a driver (done)')
sleep(2)

# Import username and password
credential = open('credentials.txt')
line = credential.readlines()
username = line[0]
password = line[1]
print('Importing the login credentials (done)')
sleep(2)

# Input the login credentials
email_field = driver.find_element_by_id('username')
email_field.send_keys(username)
print('Input email (done)')
sleep(3)

password_field = driver.find_element_by_name('session_password')
password_field.send_keys(password)
print('Input password (done)')
sleep(2)

# Click the Login button
signin_field = driver.find_element_by_xpath('//*[@id="organic-div"]/form/div[3]/button')
signin_field.click()
sleep(3)

print('Login to Linkedin (done)')

Initializing a driver (done)
Importing the login credentials (done)
Input email (done)
Input password (done)
Login to Linkedin (done)


In [30]:
# Search for the profiles we want to scrape

# Locate the search bar element
search_field = driver.find_element_by_xpath('//*[@class="search-global-typeahead__input always-show-placeholder"]')

# Input the search query to the search bar
search_query = input('What profile do you want to scrape? ')
search_field.send_keys(search_query)

# Search
search_field.send_keys(Keys.RETURN)

print('Search for profiles (done)')

What profile do you want to scrape? developer
Search for profiles (done)


In [32]:
# Scrape the URLs of the profiles

# A function to extract the URLs of one page
def GetURL():
    page_source = BeautifulSoup(driver.page_source)
    profiles = page_source.find_all('a', class_ = 'app-aware-link')
    all_profile_URL = []
    for profile in profiles:
        profile_URL = profile.get('href')
        if "https://www.linkedin.com/in/" in profile_URL and "/recent-activity/" not in profile_URL:
            if profile_URL not in all_profile_URL:
                all_profile_URL.append(profile_URL)
    return all_profile_URL

# Navigate through many pages, and extract the profile URLs of each page

input_page = int(input('How many pages you want to scrape: '))
URLs_all_page = []

if(input_page > 1):
    for page in range(input_page):
        URLs_one_page = GetURL()
        sleep(3)
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') #scroll to the end of the page
        sleep(4)
        next_button = driver.find_element_by_class_name('artdeco-pagination__button--next')
        driver.execute_script("arguments[0].click();", next_button)
        URLs_all_page = URLs_all_page + URLs_one_page
        sleep(5)
else:
    URLs_one_page = GetURL()
    sleep(2)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') #scroll to the end of the page
    sleep(3)
    URLs_all_page = URLs_all_page + URLs_one_page
    sleep(2)
print(URLs_all_page)
print('Scrape the URLs (done)')

How many pages you want to scrape: 20
['https://www.linkedin.com/in/oussama-laghouen-a77a85161', 'https://www.linkedin.com/in/ACoAAAQ7-LIB-AZ1CnntwEBWSBIv1S-cPw09Al8', 'https://www.linkedin.com/in/ACoAAAXAC_MBwDvCjUHx1tU-ugeRnqEKsJCASrw', 'https://www.linkedin.com/in/syrine-ben-ayed-886835162', 'https://www.linkedin.com/in/ACoAAANAHWEBoy_jrgzQY2AhYh3Pp-JczlTCMjw', 'https://www.linkedin.com/in/ACoAAAg_SLsBTuro1evJ5ef_8e05I3DEy1Oc-f8', 'https://www.linkedin.com/in/ACoAACNTZ-oB3-KswFpn7hKQdVI1wEyKi7Q5uIA', 'https://www.linkedin.com/in/ACoAAAahqqsBdCOAtxT3_qbNkwqdkI7NnuCURN4', 'https://www.linkedin.com/in/oussema-ben-amor-1721b8166', 'https://www.linkedin.com/in/ACoAAAM0OfMBnPGMe8OhkvOVW8Yd2hXYM3bMi3A', 'https://www.linkedin.com/in/ACoAABFH7jcBPscKqNAr96rJU-aNS8uMlQ3Tv1w', 'https://www.linkedin.com/in/mariem-abdelaali-003336198', 'https://www.linkedin.com/in/ACoAAArtVSAB7tlruVEKTZCG8DNXz3wrwySKgl8', 'https://www.linkedin.com/in/ACoAABJycxgBeFHzHPPmWdSx7GzAEXTXPM6xydw', 'https://www.linkedi

In [36]:
# Scrape the data of 1 Linkedin profile, and write the data to a .CSV file

with open('developer_output.csv', 'w',  newline = '', encoding="utf-8") as file_output:
    headers = ['Name', 'Company', 'Location','Job Title', 'Experience','Skills', 'Certificates','URL']
    writer = csv.DictWriter(file_output, delimiter=',', lineterminator='\n',fieldnames=headers)
    writer.writeheader()
    for linkedin_URL in URLs_all_page:
        sleep(3)
        driver.get(linkedin_URL)
        print('Accessing profile: ', linkedin_URL)

        sleep(2)
        
        # Get name, company, location, profile bio
        try:
            name = driver.find_element_by_xpath("//h1[@class='text-heading-xlarge inline t-24 v-align-middle break-words']").text
        except:
            name = np.nan
        
        try:
            company = driver.find_element_by_xpath("//h2[@class='text-heading-small align-self-center flex-1 break-words']//div").text
        except:
            company = np.nan
        
        try:
            location = driver.find_element_by_xpath("//span[@class='text-body-small inline t-black--light break-words']").text
        except:
            location = np.nan
            
        try:
            profile_bio = driver.find_element_by_xpath("//div[@class='text-body-medium break-words']").text
        except:
            profile_bio = np.nan
        
        sleep(3)
        driver.execute_script("window.scrollTo(0, 1000)")
        
        sleep(5)
        
        # Get certificates
        
        try:
            certificates_button = driver.find_element_by_xpath("//button[@class='pv-profile-section__see-more-inline pv-profile-section__text-truncate-toggle']")
            certificates_button.click()
        except:
            pass
        
        try:
            certificates_list = []
            certificates = driver.find_elements_by_xpath("//div[@class='pv-profile-section__card-item']")
            for c in certificates:
                certificates_details = c.text
            
                certificate = {
                    "name": certificates_details.split('\n')[0],
                    "delivered_by": certificates_details.split('\n')[2]
                    }
                certificates_list.append(certificate)
        except:
            pass
        
        sleep(5)
        
        # Get skills
        try:
            show_more_skills_button = driver.find_element_by_xpath("//button[@class='pv-profile-section__card-action-bar pv-skills-section__additional-skills artdeco-container-card-action-bar artdeco-button artdeco-button--tertiary artdeco-button--3 artdeco-button--fluid artdeco-button--muted']")
            show_more_skills_button.click()
        except:
            pass
        
        sleep(3)
        skills_list = []
        skills = driver.find_elements_by_xpath("//span[@class='pv-skill-category-entity__name-text t-16 t-black t-bold']")
        for s in skills:
            skills_list.append(s.text)
        sleep(5)
        
        # Get Experience
        experience_list = []
    
        try:
            experiences = driver.find_elements_by_xpath("//section[@class='pv-profile-section__card-item-v2 pv-profile-section pv-position-entity ember-view']")
            for exp in experiences:
                experience_details = exp.text
            
                experience = {
                    "title": experience_details.split('\n')[0] if experience_details.split('\n')[0] else "",
                    "company": experience_details.split('\n')[2] if experience_details.split('\n')[2] else "",
                    "date": experience_details.split('\n')[4] if experience_details.split('\n')[4] else "",
                    "duration": experience_details.split('\n')[6] if experience_details.split('\n')[6] else ""
                }
                experience_list.append(experience)
        except:
            pass
        
        sleep(5)
            
            
        
        print(name)
        print(company)
        print(location)
        print(profile_bio)
        print(experience_list)
        print(skills_list)
        print(certificates_list)
        
        writer.writerow({headers[0]:name, 
                         headers[1]:company, 
                         headers[2]:location, 
                         headers[3]:profile_bio, 
                         headers[4]:experience_list,
                         headers[5]:skills_list, 
                         headers[6]:certificates_list,
                         headers[7]:linkedin_URL})
        sleep(5)
    print('Linkedin scraping (done)')

Accessing profile:  https://www.linkedin.com/in/oussama-laghouen-a77a85161
Oussama Laghouen
VERMEG for Banking & Insurance Software
Tunis, Tunisie
Software Developer at Vermeg
[{'title': 'Developer', 'company': 'VERMEG for Banking & Insurance Software', 'date': 'janv. 2019 – Aujourd’hui', 'duration': '2 ans 8 mois'}, {'title': 'Stage pfe', 'company': 'STAR ASSURANCES', 'date': 'févr. 2018 – juin 2018', 'duration': '5 mois'}]
['Java', 'PHP', 'C++', 'Oracle Database', 'SQL', 'HTML5', 'Oracle SQL Developer', 'Visual Basic .NET (VB.NET)', 'MySQL', 'jQuery', 'AJAX', 'UML', 'J2EE Web Services', 'Merise']
[{'name': 'Scrum Foundation Professional Certificate SFPC', 'delivered_by': 'CertiProf'}, {'name': 'Scrum Foundation Professional Certificate', 'delivered_by': 'CertiProf'}]
Accessing profile:  https://www.linkedin.com/in/ACoAAAQ7-LIB-AZ1CnntwEBWSBIv1S-cPw09Al8
Aziza Mzoughi
Lyance
Tunis, Tunisie
Group Director Human Resources chez Lyance & Cloud Control
[{'title': 'Director Human Resources'

Accessing profile:  https://www.linkedin.com/in/ACoAAAahqqsBdCOAtxT3_qbNkwqdkI7NnuCURN4
Peter David Jenkins
PDJ Copywriting
Édimbourg et périphérie
Helping business & personal development coaches to save time, build their brands and generate business by writing and editing solid website content for them. Freelance content writer | freelance copywriter | editor
[{'title': 'Freelance Copywriter, Content Writer and Editor', 'company': 'PDJ Copywriting Freelance', 'date': 'déc. 2018 – Aujourd’hui', 'duration': '2 ans 9 mois'}, {'title': 'Digital Marketing Analyst', 'company': '80 DAYS', 'date': 'août 2017 – août 2018', 'duration': '1 an 1 mois'}, {'title': 'Nom de l’entreprise', 'company': 'Durée totale', 'date': 'Poste', 'duration': 'Dates d’emploi'}, {'title': 'Customer service assistant', 'company': 'Tenpin', 'date': 'sept. 2011 – août 2012', 'duration': '1 an'}, {'title': 'Copywriter (Self Employed)', 'company': 'Apex Hotels', 'date': 'sept. 2009 – janv. 2012', 'duration': '2 ans 5 moi

Accessing profile:  https://www.linkedin.com/in/emna-belkhiria-77b17b180
emna belkhiria
VERMEG for Banking & Insurance Software
Monastir, Tunisie
Software Developer chez VERMEG for Banking & Insurance Software
[{'title': 'Software Developer', 'company': 'VERMEG for Banking & Insurance Software Contrat en alternance', 'date': 'oct. 2020 – Aujourd’hui', 'duration': '11 mois'}, {'title': 'Stagiaire', 'company': 'Tunisie Télécom Stage', 'date': 'mars 2020 – juin 2020', 'duration': '4 mois'}, {'title': 'Stagiaire été', 'company': 'BSB TOYOTA Stage', 'date': 'août 2018 – sept. 2018', 'duration': '2 mois'}]
['Développement de logiciels', 'Developpement web', 'Informatique', 'Leadership', 'Anglais']
[]
Accessing profile:  https://www.linkedin.com/in/siwaramairi
Siwar AMAIRI
VERMEG for Banking & Insurance Software
Manouba, Tunisie
Software Developer chez VERMEG for Banking & Insurance Software
[{'title': 'Software Developer', 'company': 'VERMEG for Banking & Insurance Software', 'date': 'oct. 2

Accessing profile:  https://www.linkedin.com/in/am%C3%A8na-chbil-5b5609135
Amèna CHBIL
DECADE
Monastir, Tunisie
Front-end web developer chez DECADE
[{'title': 'Front-end web developer', 'company': 'DECADE Temps plein', 'date': 'juil. 2019 – Aujourd’hui', 'duration': '2 ans 2 mois'}, {'title': 'Stage de fin d’études', 'company': 'DECADE Stage', 'date': 'févr. 2019 – mai 2019', 'duration': '4 mois'}, {'title': 'Consultante en télécommunications | Stage', 'company': 'Amaris Stage', 'date': 'juil. 2018 – août 2018', 'duration': '2 mois'}, {'title': 'Responsable logistique et des événements', 'company': 'ENETCOM Junior Entreprise', 'date': 'mai 2017 – juin 2018', 'duration': '1 an 2 mois'}, {'title': 'Stagiaire', 'company': 'Tunisie Télécom', 'date': 'juil. 2017', 'duration': '1 mois'}]
['Nuxt js', 'redux', 'Vue js', 'E-commerce', 'Marketing digital', 'Scrum', 'React.js', 'PHP', 'JavaScript', 'JIRA', 'Git', 'GitHub', 'API REST', 'Feuilles de style en cascade (CSS)', 'HTML', 'Bitbucket', 'Ma

Accessing profile:  https://www.linkedin.com/in/ACoAABFH7jcBPscKqNAr96rJU-aNS8uMlQ3Tv1w
Hassen Mseddi
Finlogik Inc
Montréal, Québec, Canada
Business development, IT Offshoring services, IT Team Augmentation, IT Project outsourcing ,Tradetech, Wealthtech, Regtech, Fintech
[{'title': 'Country Manager', 'company': 'Finlogik Inc CDI à temps plein', 'date': 'nov. 2017 – Aujourd’hui', 'duration': '3 ans 10 mois'}, {'title': 'Senior Business Process Analyst', 'company': 'FIS CDI à temps plein', 'date': 'juin 2015 – nov. 2017', 'duration': '2 ans 6 mois'}, {'title': 'Consultant senior', 'company': 'Planaxis | Groupaxis CDI à temps plein', 'date': 'sept. 2012 – mai 2015', 'duration': '2 ans 9 mois'}, {'title': 'Quantitative Analyst', 'company': 'Caisse de dépôt et placement du Québec (CDPQ) CDI à temps plein', 'date': 'janv. 2004 – sept. 2012', 'duration': '8 ans 9 mois'}]
['FinTech']
[]
Accessing profile:  https://www.linkedin.com/in/saidane-mossab-432077186
Saidane Mossab
Octasoft
Tunisie
Ful

...

Accessing profile:  https://www.linkedin.com/in/ACoAAANAHWEBoy_jrgzQY2AhYh3Pp-JczlTCMjw
Fehmi Omar
NETEREN | İspanyol Total Panel Kaplama
Tunis, Tunisie
Représentant exclusif
[{'title': 'Sales Representative', 'company': 'NETEREN | İspanyol Total Panel Kaplama', 'date': 'oct. 2017 – Aujourd’hui', 'duration': '3 ans 11 mois'}, {'title': 'fondateur', 'company': 'fehmidesign', 'date': 'févr. 2008 – Aujourd’hui', 'duration': '13 ans 7 mois'}]
['Management', 'Marketing', 'Microsoft Office', 'Réseaux sociaux', 'Gestion de projet', 'Stratégie marketing', 'Planification stratégique', 'Vente', 'Publicité', 'Marketing social', 'Service client', 'Leadership']
[]
Accessing profile:  https://www.linkedin.com/in/ACoAAAi1PHMBktpdlH8LR1a8ydABgaMOysy5qj0
Ali bazarbacha
Discovery Informatique SA
Tunis, Tunisie
DGA Groupe Discovery Informatique
[{'title': 'Directeur General Adjoint', 'company': 'Discovery Informatique SA Indépendant', 'date': '2004 – Aujourd’hui', 'duration': '17 ans'}, {'title': 'Chef d

Accessing profile:  https://www.linkedin.com/in/ACoAAAaf1HQBqoXIIy6Cth0anPN-b3_IzLXrYBw
Aymen Sellaouti
Institut national de sciences appliquées et de technologies (INSAT)
Tunis, Tunisie
Maître Assistant à L'INSAT, Content Creator à Techwall, Formateur et Consultant en nouvelles technologies.
[{'title': 'Maître assistant', 'company': 'Institut national de sciences appliquées et de technologies (INSAT)', 'date': 'sept. 2013 – Aujourd’hui', 'duration': '8 ans'}, {'title': 'Consultant', 'company': 'Tridevs', 'date': 'janv. 2018 – Aujourd’hui', 'duration': '3 ans 8 mois'}, {'title': 'Formation', 'company': 'Aymen Sellaouti', 'date': '2015 – Aujourd’hui', 'duration': '6 ans'}, {'title': 'Enseignant', 'company': 'Faculté des sciences de Tunis', 'date': 'sept. 2009 – Aujourd’hui', 'duration': '12 ans'}, {'title': 'Docteur', 'company': 'Unistra', 'date': '2012 – 2014', 'duration': '2 ans'}]
['C', 'C++', 'Java', 'Développement de logiciel', 'MySQL', 'UML', 'JavaScript', 'Linux', 'CSS', 'PHP', '

Accessing profile:  https://www.linkedin.com/in/sihem-nasraoui-190324185
Sihem Nasraoui
be-softilys Tunisie
Tunis, Tunis, Tunisie
Full Stack Developer (Spring boot, Angular) at be-softilys Tunisie
[{'title': 'Nom de l’entreprise', 'company': 'Durée totale', 'date': 'Poste', 'duration': 'Temps plein'}, {'title': 'Cultural-artistic group Kasserine-Madrid Coordinator', 'company': 'ConArte International', 'date': 'août 2018 – janv. 2020', 'duration': '1 an 6 mois'}, {'title': 'Peace Lab Focal Point', 'company': 'Young Leaders Entrepreneurs', 'date': 'mars 2017 – janv. 2020', 'duration': '2 ans 11 mois'}, {'title': 'Secretary General', 'company': 'Jeunes Ingénieurs ISAMM', 'date': 'sept. 2018 – sept. 2019', 'duration': '1 an 1 mois'}, {'title': 'Engineer Intern', 'company': 'GoodLinks', 'date': 'juil. 2019 – août 2019', 'duration': '2 mois'}]
['Leadership', 'Time Management', 'Microsoft Office', 'Web Development', 'Mobile Applications', 'Front-end Development', 'Laravel', 'php', 'PhpMyAdmin

Accessing profile:  https://www.linkedin.com/in/ACoAAAA9PusBFaVbV20Kin9CiKT3xUnzc-EnVZ8
Menno Mafait [discovered natural intelligence]
Thinknowlogy
Pays-Bas
Based on natural intelligence, Thinknowlogy is the world's most advanced – and the world's only multilingual – reasoner.
[{'title': "Thinknowlogy is based on natural intelligence • results that scientists can't deliver • open source", 'company': 'Thinknowlogy', 'date': 'juil. 2007 – Aujourd’hui', 'duration': '14 ans 2 mois'}, {'title': 'Software Tester', 'company': 'Orange Broadband', 'date': 'oct. 2006 – juin 2007', 'duration': '9 mois'}, {'title': 'Software Tester', 'company': 'ProRail', 'date': 'mars 2006 – sept. 2006', 'duration': '7 mois'}, {'title': 'Software Test Consultant', 'company': 'Atos Origin Nederland BV', 'date': 'août 2005 – sept. 2006', 'duration': '1 an 2 mois'}, {'title': 'Software Tester', 'company': 'Achmea', 'date': 'août 2005 – févr. 2006', 'duration': '7 mois'}]
['Artificial Intelligence', 'Natural Language

Accessing profile:  https://www.linkedin.com/in/ACoAAATXqG0BfjtdOXnyPlArUbcWZklN6dxdR54
Aymen Mbarek
IntilaQ Tunisia
Soukra, Ariana, Tunisie
Early stage investment expert
[{'title': 'Investment and support manager', 'company': 'IntilaQ Tunisia', 'date': 'déc. 2014 – Aujourd’hui', 'duration': '6 ans 9 mois'}, {'title': 'Senior financial analyst (Early stage funding)', 'company': 'Alternative Capital Partners', 'date': 'avr. 2013 – nov. 2014', 'duration': '1 an 8 mois'}, {'title': 'Financial analyst (Early stage funding)', 'company': 'Sages Capital', 'date': 'oct. 2010 – mars 2013', 'duration': '2 ans 6 mois'}, {'title': 'Head of spin-off department', 'company': 'SNCPA', 'date': 'févr. 2007 – déc. 2009', 'duration': '2 ans 11 mois'}]
[]
[]
Accessing profile:  https://www.linkedin.com/in/ahmed19chebil
Ahmed Chebil
ARSELA
Jamel, Monastir, Tunisie
MERN developer at ARSELA
[{'title': 'Web Developer', 'company': 'ARSELA Temps plein', 'date': 'août 2020 – Aujourd’hui', 'duration': '1 an 1 mois

In [8]:
import pandas as pd
data_frontend = pd.read_csv('frontend_output.csv')
data_frontend.shape

(250, 8)

In [9]:
data_backend = pd.read_csv('backend_output.csv')
data_backend.shape

(104, 8)

In [10]:
data_datasc = pd.read_csv('datasc_output.csv')
data_datasc.shape

(249, 8)

In [18]:
data_iot = pd.read_csv('iot_output.csv')
data_iot.shape

(118, 8)

In [38]:
data_developer = pd.read_csv('developer_output.csv')
data_developer.shape

(473, 8)

In [39]:
df = pd.DataFrame()

In [40]:
df = df.append(data_frontend)
df = df.append(data_backend)
df = df.append(data_datasc)
df = df.append(data_iot)
df = df.append(data_developer)
df.shape

(1194, 8)

In [41]:
# df = df.drop(['Experience'], axis=1)
df = df.drop_duplicates()

In [42]:
df.shape

(701, 8)